# Frontier Exploration Assignment

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/AIResearchLab/foundations-of-robotics-labs/blob/master/2-navigation/02-nav-assignment.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/AIResearchLab/foundations-of-robotics-labs/master?filepath=2-navigation/02-nav-assignment.ipynb)

In this assignment, you will implement a frontier-based exploration algorithm. The algorithm will be tested in a simulated environment using the Gazebo simulator.

### What is a frontier?

A frontier is a line that separates explored and unexplored areas.

### What is a frontier-based exploration algorithm?

A frontier-based exploration algorithm is an algorithm that uses frontiers to explore an unknown environment. The algorithm works as follows:

1. Find a frontier.
2. Move the robot to the frontier.
3. Repeat.

#### How to find a frontier?

To find a frontier, you need to find a line that separates explored and unexplored areas. To do this, you need to know which areas are explored and which are unexplored. You can get this information from the map. The map is a grid that represents the environment. Each cell in the grid is either occupied or free. If a cell is not free or occupied it is unexplored space that the robot needs to explore.

## How to complete the assignment

To complete the assignment, you need to implement the `find_frontier` method in the `FrontierExplorer` class. The function takes a map as input and returns a frontier.

The `FrontierExplorer` class must also have a `explore` function that calls the `find_frontier` method, moves the robot to the frontier and repeats. Moving the robot is done using *Actions* and is implemented for you.

### How to test your code

Run the ROS launch file to start the Gazebo simulator and the RViz visualizer.

```bash
roslaunch sphero_rvr_algorithms ball_maze.launch
```

In [ ]:
%%bash --bg
roslaunch sphero_rvr_algorithms ball_maze.launch

**Then run the main function below to test your code.**

In [ ]:
"""
Frontier exploration assignment

Implement a frontier exploration algorithm
"""


import rospy
import actionlib
from nav_msgs.msg import OccupancyGrid
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
import numpy as np
import tf2_ros


class FrontierExplorer(object):
    def __init__(self, rate=1.0):
        """
        Class constructor
        """
        # create an exploration timer to explore the environment every 1/rate seconds
        self.rate = rate
        self.exploration_timer = rospy.Timer(rospy.Duration(1.0 / self.rate), self.explore_timer_cb, oneshot=True)

        # subscribe to the occupancy grid
        self.occupancy_grid_sub = rospy.Subscriber('/map', OccupancyGrid, self.occupancy_grid_cb, queue_size=1)
        self.occupancy_grid = None

        # create a move base client to send goals to move_base
        self.move_base_client = actionlib.SimpleActionClient('/move_base', MoveBaseAction)
        self.move_base_client.wait_for_server()

        # create a tf buffer and listener
        self.tf_buffer = tf2_ros.Buffer()
        self.tf_listener = tf2_ros.TransformListener(self.tf_buffer)

        '''
        your code here
        '''

    def find_frontier(self, occupancy_grid):
        """
        find the frontier cells in the map

        Args:
            occupancy_grid (numpy.array): occupancy grid map of the environment

        Returns:
            (list): list containing the frontier cells in the map
        """

        # empty frontier cells list
        frontier_cells = []

        # empty occ grid guard
        if occupancy_grid is None:
            return []

        '''
        your code here
        '''

        # return the frontier cells
        return frontier_cells

    def explore(self):
        """
        Implement a frontier exploration algorithm

        work out the frontier cells in the map
        then move to the closest frontier (challenge)
        """

        # create a copy of the occupancy grid
        occ = self.occupancy_grid.copy()

        # find the frontier cells
        frontier_cells = self.find_frontier(occ)

        # the selected frontier to move toward
        selected_frontier_cell = None

        '''
        (challenge)
        find the nearest frontier to the robot

        (hint)
        use the robot's pose in the 'map' frame of reference to find the nearest frontier
        use the tf buffer and listener to get the robot's pose in the 'map' frame of reference

        your code here
        '''

        # no selected frontier cell guard
        if selected_frontier_cell is None:
            return

        # move to the selected frontier cell
        self.move_base_to_frontier(selected_frontier_cell)

    ## helper functions
    # state timer callback
    def explore_timer_cb(self, event):
        # do the exploration method
        self.explore()

        # restart the timer
        self.exploration_timer = rospy.Timer(rospy.Duration(1.0 / self.rate), self.explore_timer_cb, oneshot=True)

    # occupancy grid callback
    def occupancy_grid_cb(self, msg):
        # convert the occupancy grid message to a numpy array
        self.occupancy_grid = np.array(msg.data).reshape((msg.info.height, msg.info.width))

    # move to selected frontier using move_base
    def move_base_to_frontier(self, coord):
        # create a move base goal message
        goal = MoveBaseGoal()
        goal.target_pose.header.frame_id = 'map'
        goal.target_pose.header.stamp = rospy.Time.now()

        # set the goal position
        goal.target_pose.pose.position.x = coord[0]
        goal.target_pose.pose.position.y = coord[1]
        goal.target_pose.pose.position.z = 0.0

        # send the goal
        self.move_base_client.send_goal(goal)

        # wait for the goal to complete
        self.move_base_client.wait_for_result()

# main function
def main():
    # initialize a node
    rospy.init_node('frontier_explorer')

    # create a frontier explorer object
    frontier_explorer = FrontierExplorer()

    # spin
    rospy.spin()


---

#### Main Function

**Run the following cell to test your code.**

> Note:
>
> Don't forget to run the cell that defines the `FrontierExplorer` class before running the main function, and after any changes you make to the class.

In [ ]:
# run main
main()